In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet as wn

In [2]:
wsd_test_filename = "../data/wsd_contexts.txt"
brown_corpus = brown.tagged_sents()

In [13]:
senses_list = []
allWords ={}
with open(wsd_test_filename, 'r') as wsd_file:
    for line in wsd_file:
        line = line.strip('\n')
        line = line.split('\t')
        probe_word = line[0]
        noun_groups = line[1].split(',')

        for ws in wn.synsets(probe_word, pos = wn.NOUN):
            senses_list.append(ws)
            for context in noun_groups:
                for cs in wn.synsets(context, pos = wn.NOUN):
                    senses_list.append(cs)
                    subsumers = ws.common_hypernyms(cs)
                    for subsumer in subsumers:
                        if subsumer._pos == 'n':
                            senses_list.append(subsumer)

        for sense in senses_list:
            lemma, pos, index = sense._name.split('.')
            allWords[lemma] = None

In [15]:
for aw in allWords:
    allWords[aw] = 1

In [31]:
tags ={}
for sent in brown_corpus:
    for w,t in sent:
        if t in tags:
            tags[t]+=1
        else:
            tags[t] = 1
        if w in allWords and t =='NN':
            allWords[w] +=1

In [23]:
total_words = sum(allWords.values())

In [30]:
import math
with open('custom_ic.dat', 'w') as newIcFile:

    # not quite sure what this does

    newIcFile.write('wnver::eOS9lXC6GvMWznF1wkZofDdtbBU' + '\n')
    newIcFile.write('1740n ' + str(total_words) + ' ROOT\n')
    
    for word in allWords:

        synsets = wn.synsets(word)
        count = allWords[word]
        total = total_words
        logProb = -math.log(float(count) / total)
        
        for synset in synsets:
            line = str(synset._offset) + 'n' + ' ' + str(logProb) + '\n'            
            newIcFile.write(line)
            

In [35]:
with open('tags.txt','w') as t_file:
    for t in tags.items():
        t_file.write(t[0]+"\n")